In [1]:
%env SPARK_VERSION=3.0.0# TODO PUT YOUR VALUE

env: SPARK_VERSION=3.0.0# TODO PUT YOUR VALUE


In [5]:
pip install pydeequ

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession, Row
import pydeequ

spark = (SparkSession
    .builder
    .appName("Connection through JDBC")\
    .config("spark.driver.extraClassPath","/home/jovyan/work/sqljdbc42.jar") \
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())


df = spark.sparkContext.parallelize([
            Row(a="foo", b=1, c=5),
            Row(a="bar", b=2, c=6),
            Row(a="baz", b=3, c=None)]).toDF()

In [72]:
import pyodbc

'3.4.0'

In [3]:
server_url = "jdbc:sqlserver://EPPLGDAW00A5\SQLEXPRESS:1433" # think what should be here
database_name = "AdventureWorks2016"
url = server_url + ";" + "databaseName=" + database_name + ";"


table = "HumanResources.Department"
user = "nastia" # your user creds here
password  = "AMSTERdam19" # your user creds here

df_department=spark.read \
    .format("jdbc") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .option("url", f"jdbc:sqlserver://EPPLGDAW00A5\SQLEXPRESS:1433;databaseName={database_name}") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .load()

df_department.show(5)

# TODO connect to DB with Spark using JDBC connection to read the data

+------------+-----------+--------------------+-------------------+
|DepartmentID|       Name|           GroupName|       ModifiedDate|
+------------+-----------+--------------------+-------------------+
|           1|Engineering|Research and Deve...|2008-04-30 00:00:00|
|           2|Tool Design|Research and Deve...|2008-04-30 00:00:00|
|           3|      Sales| Sales and Marketing|2008-04-30 00:00:00|
|           4|  Marketing| Sales and Marketing|2008-04-30 00:00:00|
|           5| Purchasing|Inventory Management|2008-04-30 00:00:00|
+------------+-----------+--------------------+-------------------+
only showing top 5 rows



In [7]:
### Data Analyzers section
# TODO analyze data here
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(df_department) \
                    .addAnalyzer(Completeness("DepartmentID")) \
                    .addAnalyzer(ApproxCountDistinct("DepartmentID")) \
                    .addAnalyzer(Mean("DepartmentID")) \
                    .addAnalyzer(Completeness("GroupName")) \
                    .addAnalyzer(Size()) \
                    .run()

analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

+-------+------------+-------------------+-----+
| entity|    instance|               name|value|
+-------+------------+-------------------+-----+
| Column|DepartmentID|       Completeness|  1.0|
| Column|DepartmentID|ApproxCountDistinct| 16.0|
| Column|DepartmentID|               Mean|  8.5|
|Dataset|           *|               Size| 16.0|
| Column|   GroupName|       Completeness|  1.0|
+-------+------------+-------------------+-----+



In [27]:
### Data profiling section
# TODO profile data here
from pydeequ.profiles import *

result = ColumnProfilerRunner(spark) \
    .onData(df_department) \
    .run()

#for col, profile in result.profiles.items():
 #   print(profile)
for col, profile in result.profiles.items():
    print(profile)

NumericProfiles for column: DepartmentID: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 16,
    "dataType": "Integral",
    "isDataTypeInferred": false,
    "typeCounts": {},
    "histogram": [
        [
            "12",
            1,
            0.0625
        ],
        [
            "8",
            1,
            0.0625
        ],
        [
            "4",
            1,
            0.0625
        ],
        [
            "15",
            1,
            0.0625
        ],
        [
            "11",
            1,
            0.0625
        ],
        [
            "9",
            1,
            0.0625
        ],
        [
            "13",
            1,
            0.0625
        ],
        [
            "16",
            1,
            0.0625
        ],
        [
            "5",
            1,
            0.0625
        ],
        [
            "10",
            1,
            0.0625
        ],
        [
            "6",
            1,
            0.0625
  

In [28]:
 # better view profile output
    
for col, profile in result.profiles.items():
       print(f'Column \'{col}\'')
       print('\t',f'completeness: {profile.completeness}')
       print('\t',f'approximate number of distinct values: {profile.approximateNumDistinctValues}')
       print('\t',f'datatype: {profile.dataType}') 

Column 'DepartmentID'
	 completeness: 1.0
	 approximate number of distinct values: 16
	 datatype: Integral
Column 'Name'
	 completeness: 1.0
	 approximate number of distinct values: 16
	 datatype: String
Column 'GroupName'
	 completeness: 1.0
	 approximate number of distinct values: 6
	 datatype: String
Column 'ModifiedDate'
	 completeness: 1.0
	 approximate number of distinct values: 1
	 datatype: String


In [6]:
### Constraint Suggestions section
# TODO find meaninful constraints here
from pydeequ.suggestions import *

suggestionResult = ConstraintSuggestionRunner(spark) \
             .onData(df_department) \
             .addConstraintRule(DEFAULT()) \
             .run()

# Constraint Suggestions in JSON format
#print(suggestionResult)
print(json.dumps(suggestionResult, indent=2))

{
  "constraint_suggestions": [
    {
      "constraint_name": "ComplianceConstraint(Compliance('GroupName' has value range 'Executive General and Administration', 'Research and Development', 'Quality Assurance', 'Sales and Marketing', 'Inventory Management', 'Manufacturing',`GroupName` IN ('Executive General and Administration', 'Research and Development', 'Quality Assurance', 'Sales and Marketing', 'Inventory Management', 'Manufacturing'),None))",
      "column_name": "GroupName",
      "current_value": "Compliance: 1",
      "description": "'GroupName' has value range 'Executive General and Administration', 'Research and Development', 'Quality Assurance', 'Sales and Marketing', 'Inventory Management', 'Manufacturing'",
      "suggesting_rule": "CategoricalRangeRule()",
      "rule_description": "If we see a categorical range for a column, we suggest an IS IN (...) constraint",
      "code_for_constraint": ".isContainedIn(\"GroupName\", [\"Executive General and Administration\", \"Re

In [23]:
### Constraint Verification section
# TODO check selected constraints here and make beautify the report
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "Review Check")

checkResult = VerificationSuite(spark) \
                  .onData(df_department) \
                  .addCheck(
                    check.hasSize(lambda x: x >= 16)
                  .hasMin("DepartmentID", lambda x: x == 1) \
                  .hasMax("DepartmentID", lambda x: x == 16) \
                  .isNonNegative("DepartmentID") \
                  .isComplete("DepartmentID") \
                  .isComplete("Name") \
                  .isComplete("GroupName") \
                  .isComplete("ModifiedDate") \
                  .isUnique("DepartmentID")  \
                  .isUnique("Name")  \
                  .isContainedIn("GroupName", ["Executive General and Administration", "Inventory Management", "Manufacturing", "Quality Assurance", "Research and Development", "Sales and Marketing"]) \
                  ) \
                  .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+------------+-----------+------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+------------------+
|check       |check_level|check_status|constraint                                                                                                                                                                                                                                                                                                                                                                                     |constraint_status|constraint_message|
+------------+-----------+------------+---------------------------------------

In [25]:
import pandas as pd

result=checkResult_df.toPandas().to_csv("result.csv")